## xTTS-2.v fine-tuning

In [3]:
import os
from trainer import Trainer, TrainerArgs
from TTS.config.shared_configs import BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.layers.xtts.trainer.gpt_trainer import GPTArgs, GPTTrainer, GPTTrainerConfig, XttsAudioConfig
from TTS.utils.manage import ModelManager

In [4]:
# 로깅 파라미터
RUN_NAME = "GPT_XTTS_v2.0_BBANGHYONG_FT"
PROJECT_NAME = "XTTS_trainer"
DASHBOARD_LOGGER = "tensorboard"
LOGGER_URI = None

# 체크포인트가 저장될 경로를 설정합니다. 기본값: ./run/training/
OUT_PATH = os.path.join(os.getcwd(), "run", "training")

In [5]:
# 훈련 파라미터
OPTIMIZER_WD_ONLY_ON_WEIGHTS = True  # 멀티 GPU 훈련의 경우 False로 설정하세요
START_WITH_EVAL = True  # True로 설정하면 평가부터 시작합니다
BATCH_SIZE = 3  # 배치 크기를 설정하세요
GRAD_ACUMM_STEPS = 84  # 그래디언트 누적 단계를 설정하세요

In [6]:
# 파인튜닝에 사용할 데이터셋을 여기에 정의하세요.
config_dataset = BaseDatasetConfig(
    formatter="ljspeech",
    dataset_name="bbanghyong",
    path="./wavs",
    meta_file_train="../metadata.txt",
    language="ko",
)

# 데이터셋의 설정을 여기에 추가하세요
DATASETS_CONFIG_LIST = [config_dataset]

In [ ]:
# XTTS v2.0.1 파일이 다운로드될 경로를 정의하세요
CHECKPOINTS_OUT_PATH = os.path.join(OUT_PATH, "XTTS_v2.0_original_model_files/")
os.makedirs(CHECKPOINTS_OUT_PATH, exist_ok=True)

# DVAE 파일들
DVAE_CHECKPOINT_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/dvae.pth"
MEL_NORM_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/mel_stats.pth"

# 다운로드된 파일의 경로를 설정하세요
DVAE_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(DVAE_CHECKPOINT_LINK))
MEL_NORM_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(MEL_NORM_LINK))

# DVAE 파일이 필요한 경우 다운로드합니다
if not os.path.isfile(DVAE_CHECKPOINT) or not os.path.isfile(MEL_NORM_FILE):
    print(" > DVAE 파일을 다운로드합니다!")
    ModelManager._download_model_files([MEL_NORM_LINK, DVAE_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=True)

# 필요한 경우 XTTS v2.0 체크포인트를 다운로드합니다
TOKENIZER_FILE_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/vocab.json"
XTTS_CHECKPOINT_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/model.pth"

# XTTS 전이 학습 파라미터: 파인튜닝할 XTTS 모델 체크포인트의 경로를 제공해야 합니다.
TOKENIZER_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(TOKENIZER_FILE_LINK))  # vocab.json 파일
XTTS_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(XTTS_CHECKPOINT_LINK))  # model.pth 파일

# 필요한 경우 XTTS v2.0 파일을 다운로드합니다
if not os.path.isfile(TOKENIZER_FILE) or not os.path.isfile(XTTS_CHECKPOINT):
    print(" > XTTS v2.0 파일을 다운로드합니다!")
    ModelManager._download_model_files(
        [TOKENIZER_FILE_LINK, XTTS_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=True
    )


 > DVAE 파일을 다운로드합니다!


100%|██████████| 1.07k/1.07k [00:00<00:00, 1.72kiB/s]


 > XTTS v2.0 파일을 다운로드합니다!


100%|██████████| 211M/211M [00:03<00:00, 53.3MiB/s]
100%|██████████| 361k/361k [00:01<00:00, 346kiB/s] 


In [9]:
# 훈련 문장 생성
SPEAKER_REFERENCE = [
    "./wavs/audio2.wav"  # 훈련 테스트 문장에서 사용할 화자 참조
]
LANGUAGE = config_dataset.language

def main():
    # args와 config 초기화
    model_args = GPTArgs(
        max_conditioning_length=132300,  # 6초
        min_conditioning_length=66150,  # 3초
        debug_loading_failures=False,
        max_wav_length=255995,  # 약 11.6초
        max_text_length=200,
        mel_norm_file=MEL_NORM_FILE,
        dvae_checkpoint=DVAE_CHECKPOINT,
        xtts_checkpoint=XTTS_CHECKPOINT,  # 파인튜닝할 모델의 체크포인트 경로
        tokenizer_file=TOKENIZER_FILE,
        gpt_num_audio_tokens=1026,
        gpt_start_audio_token=1024,
        gpt_stop_audio_token=1025,
        gpt_use_masking_gt_prompt_approach=True,
        gpt_use_perceiver_resampler=True,
    )
    # 오디오 설정 정의
    audio_config = XttsAudioConfig(sample_rate=22050, dvae_sample_rate=22050, output_sample_rate=24000)
    # 훈련 파라미터 설정
    config = GPTTrainerConfig(
        output_path=OUT_PATH,
        model_args=model_args,
        run_name=RUN_NAME,
        project_name=PROJECT_NAME,
        run_description="""
            GPT XTTS 훈련
            """,
        dashboard_logger=DASHBOARD_LOGGER,
        logger_uri=LOGGER_URI,
        audio=audio_config,
        batch_size=BATCH_SIZE,
        batch_group_size=48,
        eval_batch_size=BATCH_SIZE,
        num_loader_workers=8,
        eval_split_max_size=256,
        print_step=50,
        plot_step=100,
        log_model_step=1000,
        save_step=10000,
        save_n_checkpoints=1,
        save_checkpoints=True,
        # 목표 손실 설정 "loss"
        print_eval=False,
        # tortoise와 유사한 Optimizer 값, non-weight 파라미터에 WD를 적용하지 않도록 수정된 PyTorch 구현
        optimizer="AdamW",
        optimizer_wd_only_on_weights=OPTIMIZER_WD_ONLY_ON_WEIGHTS,
        optimizer_params={"betas": [0.9, 0.96], "eps": 1e-8, "weight_decay": 1e-2},
        lr=5e-06,  # 학습률
        lr_scheduler="StepLR",
        # 새로운 단계 체계에 맞게 조정됨
        lr_scheduler_params={"step_size": 50, "gamma": 0.5, "last_epoch": -1},
        test_sentences=[
            {
                "text": "나에게는 그들보다 이 점등인이 더 나은 사람이야. 적어도 점등인은 그들과는 달리, 남을 위해 일하기 때문이야. 너는 나에게 이 세상에 단 하나뿐인 존재가 되는 거고, 나도 너에게 세상에 하나뿐인 존재가 되는 거야.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "넌 네가 길들인 것에 영원히 책임이 있어. 누군가에게 길들여진다는 것은 눈물을 흘릴 일이 생긴다는 뜻일지도 몰라. 네 장미꽃이 너의 장미꽃이 되려면 너는 그녀를 길들여야 해. 그렇지 않으면 그녀에게는 오직 하나의 장미꽃일 뿐일 거야.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "정말로 내가 네게 필요한 것은 네가 가지고 있는 무기가 아니야. 내가 네게서 정말로 필요한 건 네가 수없이 열어준 문이야.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "너는 정말로 내게 신경 쓰지 않아. 또한 나에게 필요한 건 없고 또한 나의 많은 것을 나누기로 결정했어. 그냥 어떤 경우에나 내 행복을 위해 너를 원하지 않아. 나는 그런 사랑을 받고 싶어하지 않았어.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "난 당신이 아주 잘못 생각하고 있다고 생각해. 그러나 너는 나에게 더 많은 도움이 되고, 그렇게 도움이 필요할 때 마다 그들에게 전달 할 수있을 것입니다. 그렇지 않으면 그녀의 사랑과 나의 정확한 것을 말한다.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "오늘날의 전세계 사람들이 모두 다른 곳에 살고 있어요. 그들은 모두 각자의 길을 걷고 있고, 모두 자신이 갈 곳을 알고 있어요. 그리고 그들은 그 길을 지나 살고 있어요.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "사실 그는 전체 원인을 보자고 그는 말하고, 그는 내게 얘기 했다. 그는 내게 말하고, 나는 내가 그렇게했다.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "하루하루 시간이 되면 우리는 더 나은 미래를 찾기 위해 더 큰 노력을 계속할 수 있습니다. 또한 우리는 더 나은 시간을 보낼 것이고, 더 나은 미래를 만들 수 있습니다.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "이제는 모든 시간이 지나도 우리는 더 많은 노력을 기울여야 할 것이다. 또한 우리는 더 많은 노력을 기울여야 할 것이다.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "현대 인류는 여행하는 중이며, 이곳을 탐험하는 중이며, 이 곳에서 살고 있습니다. 우리는 모두 각자의 방법으로 각자의 방법을 찾고, 그 방법을 갈 것입니다.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "당신의 여행이 끝날 때까지 당신의 여행은 여행이 될 것입니다. 여행이 끝나지 않을 때, 우리는 여행을 계속할 것입니다.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "당신의 집을 잃어버린다면, 당신은 당신의 집을 잃을 것입니다. 당신의 집을 잃어버린다면, 당신은 당신의 집을 잃을 것입니다.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "다른 나라에 살고 있습니다. 그리고 우리는 다른 사람들에게 기여할 것입니다.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "당신은 세계를 탐험하는 중입니다. 그리고 우리는 모두가 그것을 탐험하고 있습니다. 그리고 그것을 탐험하고 있습니다.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "당신은 당신의 모든 것을 잃어 버렸습니다. 그리고 우리는 그것을 찾을 수 있습니다.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
        ],
    )

    # 설정에서 모델 초기화
    model = GPTTrainer.init_from_config(config)

    # 훈련 샘플 로드
    train_samples, eval_samples = load_tts_samples(
        DATASETS_CONFIG_LIST,
        eval_split=True,
        eval_split_max_size=config.eval_split_max_size,
        eval_split_size=config.eval_split_size,
    )

    # 트레이너 초기화 및 실행
    trainer = Trainer(
        TrainerArgs(
            restore_path=None,  # xtts_checkpoint 키를 통해 xtts 체크포인트가 복원되므로 Trainer의 restore_path 파라미터를 사용해 복원할 필요가 없음
            skip_train_epoch=False,
            start_with_eval=START_WITH_EVAL,
            grad_accum_steps=GRAD_ACUMM_STEPS,
        ),
        config,
        output_path=OUT_PATH,
        model=model,
        train_samples=train_samples,
        eval_samples=eval_samples,
    )
    trainer.fit()


if __name__ == "__main__":
    main()


>> DVAE weights restored from: d:\GitHub\AI\LangChain\xTTS\run\training\XTTS_v2.0_original_model_files/dvae.pth
 | > Found 332 files in D:\GitHub\AI\LangChain\xTTS\wavs


 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: False
 | > Precision: float32
 | > Num. of CPUs: 8
 | > Num. of Torch Threads: 1
 | > Torch seed: 1
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=d:\GitHub\AI\LangChain\xTTS\run\training\GPT_XTTS_v2.0_BBANGHYONG_FT-June-24-2024_11+36PM-ad82aa3f

 > Model has 518442047 parameters

 > EPOCH: 0/1000
 --> d:\GitHub\AI\LangChain\xTTS\run\training\GPT_XTTS_v2.0_BBANGHYONG_FT-June-24-2024_11+36PM-ad82aa3f


 > Filtering invalid eval samples!!
 > Total eval samples after filtering: 0


PermissionError: [WinError 32] 다른 프로세스가 파일을 사용 중이기 때문에 프로세스가 액세스 할 수 없습니다: 'd:/GitHub/AI/LangChain/xTTS/run/training/GPT_XTTS_v2.0_BBANGHYONG_FT-June-24-2024_11+36PM-ad82aa3f\\trainer_0_log.txt'